In [14]:
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import google.auth
from googleapiclient.discovery import build

In [ ]:
creds, _ = google.auth.default()
service = build("calendar", "v3", credentials=creds)

excel_path = "/home/jumpei_private/workspace/熊猫カンパニー/reservation/予約管理.xlsx"
reservation_df = pd.read_excel(excel_path, sheet_name="reservations_2026")
reservation_df = reservation_df[reservation_df["カレンダー登録"] != "済"]
tz = ZoneInfo("Asia/Tokyo")
for idx, row in reservation_df.iterrows():
    name = str(row["名前"])
    reservation_no = str(row["予約番号"])
    facility = str(row["公園・施設"])
    use_day_raw = str(row["利用日"])
    start_raw = str(row["開始"])

    # ---- parse date ----
    month = int(use_day_raw.split("月")[0])
    day = int(use_day_raw.split("月")[1].split("日")[0])

    hour = int(start_raw.split("時")[0])
    minute = int(start_raw.split("時")[1].split("分")[0])

    start_dt = datetime(year, month, day, hour, minute, tzinfo=tz)
    end_dt = start_dt + timedelta(hours=2)

    event = {
        "summary": f"熊猫 @{facility}",
        "location": facility,
        "description": (
            f"予約番号: {reservation_no}\n"
            f"ID: {row['userid']}\n"
            f"利用者: {name}"
        ),
        "start": {"dateTime": start_dt.isoformat()},
        "end": {"dateTime": end_dt.isoformat()},
        "colorId": "5",
    }

    created = service.events().insert(
        calendarId="primary",
        body=event
    ).execute()

    reservation_df.loc[idx, "カレンダー登録"] = "済"

created: https://www.google.com/calendar/event?eid=OWNqNGpjc2M0dmhudmJocWpybDBuYTN2a3Mgay5qdW1wZWkwOTE1QG0
created: https://www.google.com/calendar/event?eid=M3ZuanJlMGU5MW1waHRudmFiaDQ0M2pvZWcgay5qdW1wZWkwOTE1QG0
created: https://www.google.com/calendar/event?eid=cTQzOTEybTl1Y2o5OGVqcmlrZ2x2dG1xNWsgay5qdW1wZWkwOTE1QG0
created: https://www.google.com/calendar/event?eid=bTFxMzJjNXE4cDUxdmFpam03NW1rY2FvNGcgay5qdW1wZWkwOTE1QG0
created: https://www.google.com/calendar/event?eid=Z2tiM3R0ZmZsY2E5YXM4aHIyN3FrcXJrbjggay5qdW1wZWkwOTE1QG0


In [16]:
reservation_df.to_excel(excel_path, sheet_name="reservations_2026", index=False)